In [1]:
%gui qt
%matplotlib qt
import xmcd_projection
from xmcd_projection import deep_reload
deep_reload(xmcd_projection)
from xmcd_projection import *
import matplotlib.pyplot as plt
from glob import glob
import os
from skimage.io import imread, imsave
from tqdm import tqdm
from time import time

### Get file paths

In [15]:
folder_path = r"D:\Magnumfe\al6-1-1_peem_profile2"

data_folder = os.path.join(folder_path, "data")
mag_files = glob(os.path.join(data_folder, "*.csv"))
raytracing_file = os.path.join(data_folder, "raytracing.npy")

# sort mag files
# indx = [int(mf.split('.')[-2]) for mf in mag_files]
indx = [int(mf.split('_')[-1].split('.')[0]) for mf in mag_files]
mag_files = [mf for _, mf in sorted(zip(indx, mag_files))]

## If raytracing generated

In [3]:
# load raytracing if exists
raytr = np.load(raytracing_file, allow_pickle=True).item()
struct = raytr.struct
struct_projected = raytr.struct_projected

## Generate raytracing - skip if generated

In [ ]:
# need to have a mesh file
msh_file = glob(os.path.join(data_folder, "*.msh"))[0]
# get the mesh
msh = Mesh.from_file(msh_file)

#### Make sure that the projection vector is correct and that the structure is oriented well

In [ ]:
# get the projection vector
p = get_projection_vector(90, 16)

# prepare raytracing object
raytr = RayTracing(msh, p)
struct = raytr.struct
struct_projected = raytr.struct_projected

In [5]:
vis = MeshVisualizer(struct, struct_projected)
vis.show()

#### Get the ray tracing information and save

In [ ]:
raytr.get_piercings()
np.save(raytracing_file, raytr, allow_pickle=True)

## Get the xmcd

#### Get the magnetisation from csv files
Note: sometimes if the mesh file has multiple parts, the paraview export and the original mesh coordinates are not in the same order. I add a function to fix that when necessary

In [8]:
magnetisation, mag_points = load_mesh_magnetisation(mag_files[-1])
shuffle_file = os.path.join(data_folder, "shuffle_indx.npy")
try:
    shuffle_indx = np.load(shuffle_file)
except FileNotFoundError:
    print('File not found. Generating shuffle indx')
    shuffle_indx = msh.get_shuffle_indx(mag_points)
    np.save(shuffle_file, shuffle_indx)
magnetisation = magnetisation[shuffle_indx, :]

### Get the colours and XMCD values

In [9]:
xmcd_value = raytr.get_xmcd(magnetisation)
mag_colors = get_struct_face_mag_color(struct, magnetisation)

In [10]:
# deep_reload(xmcd_projection)
# from xmcd_projection import *
azi=90
center_struct = [-100, 100, 0]
dist_struct = 1e5
center_peem = [600, -5000, 0]
dist_peem = 7e5

vis = MeshVisualizer(struct, struct_projected, projected_xmcd=xmcd_value, struct_colors=mag_colors)
vis.show(azi=azi, center=center_peem, dist=dist_peem)

#### View different parts of the image separately

#### Both

In [ ]:
vis.update_colors(xmcd_value, mag_colors)
vis.view_both(azi=azi, center=center_peem, dist=dist_peem)

#### Projection

In [ ]:
vis.view_projection(azi=azi, center=center_peem, dist=dist_peem)

#### Structure

In [ ]:
vis.view_struct(azi=azi, center=center_struct, dist=dist_struct)

#### Saving one render

In [11]:
vis.view_both(azi=azi, center=center_peem, dist=dist_peem)
vis.save_render(os.path.join(folder_path, 'processed', 'shadow_final.png'))
vis.view_projection()
blurred = vis.get_blurred_image(desired_background=0.7)
imsave(os.path.join(folder_path, 'processed', 'shadow_final_blurred.png'), (blurred*255).astype(np.uint8), check_contrast=False)

vis.view_struct(azi=azi, center=center_struct, dist=dist_struct)
vis.save_render(os.path.join(folder_path, 'processed', 'structure_view.png'))

### Run for all magnetisations and save images

In [12]:
desired_background = 0.7 # if not None, adjusts the contrast so that the background is of the desired value in the blurred image

# define save directories
savedir = os.path.join(folder_path, 'processed', 'xmcd_projection')
savedir_blurred = savedir+'_blurred'
savedir_structure = os.path.join(folder_path, 'processed', 'projected_structure')
for sd in (savedir, savedir_blurred, savedir_structure):
    if not os.path.exists(sd):
        os.makedirs(sd)

# run and save
for mf in tqdm(mag_files):
#     file_num = int(mf.split('.')[-2])
    file_num = int(mf.split('_')[-1].split('.')[0])
    # get the magnetisation and xmcd
    magnetisation, _ = load_mesh_magnetisation(mf)
    magnetisation = magnetisation[shuffle_indx, :]
    xmcd_value = raytr.get_xmcd(magnetisation)
    mag_colors = get_struct_face_mag_color(struct, magnetisation)
    # update the view
    vis.update_colors(xmcd_value, mag_colors)
    vis.view_projection(center=center_peem, dist=dist_peem)
    file_name = 'step_{}.png'.format(file_num)
    image = vis.save_render(os.path.join(savedir, file_name))
    
    # save the blurred image
    blurred = img2uint(vis.get_blurred_image(desired_background=desired_background))
    imsave(os.path.join(savedir_blurred, file_name), blurred, check_contrast=False)
    
    # save the image of the structure
    vis.view_struct(center=center_struct, dist=dist_struct)
    image = vis.save_render(os.path.join(savedir_structure, file_name))

  0%|                                         | 0/42 [00:00<?, ?it/s]D:\Programs\xmcd-projection\xmcd_projection\mesh_visualisation.py:155: RuntimeWarning: invalid value encountered in true_divide
  img /= new1
100%|████████████████████████████████| 42/42 [01:29<00:00,  2.13s/it]


In [13]:
desired_background = 0.7 # if not None, adjusts the contrast so that the background is of the desired value in the blurred image

savedir = os.path.join(folder_path, 'processed', 'xmcd_projection_inv')
savedir_blurred = savedir+'_blurred'
savedir_structure = os.path.join(folder_path, 'processed', 'projected_structure_inv')
for sd in (savedir, savedir_blurred, savedir_structure):
    if not os.path.exists(sd):
        os.makedirs(sd)
        
for mf in tqdm(mag_files):
#     file_num = int(mf.split('.')[-2])
    file_num = int(mf.split('_')[-1].split('.')[0])
    magnetisation, _ = load_mesh_magnetisation(mf)
    magnetisation *= -1
    magnetisation = magnetisation[shuffle_indx, :]
    xmcd_value = raytr.get_xmcd(magnetisation)
    mag_colors = get_struct_face_mag_color(struct, magnetisation)
    # update the view
    vis.update_colors(xmcd_value, mag_colors)
    vis.view_projection(center=center_peem, dist=dist_peem)
    file_name = 'step_{}.png'.format(file_num)
    image = vis.save_render(os.path.join(savedir, file_name))
    
    # save the blurred image
    blurred = img2uint(vis.get_blurred_image(desired_background=desired_background))
    imsave(os.path.join(savedir_blurred, file_name), blurred, check_contrast=False)
    
    # save the image of the structure
    vis.view_struct(center=center_struct, dist=dist_struct)
    image = vis.save_render(os.path.join(savedir_structure, file_name))


  0%|                                         | 0/42 [00:00<?, ?it/s]D:\Programs\xmcd-projection\xmcd_projection\mesh_visualisation.py:161: RuntimeWarning: invalid value encountered in true_divide
  img /= 1 - new0
100%|████████████████████████████████| 42/42 [01:29<00:00,  2.13s/it]


### Import the saved images and make a movie

In [18]:
folder_path = 'D:\\Magnumfe\\al6-1-1_peem_profile2_001'

In [19]:
import cv2
from tqdm import tqdm
FPS = 10

savedir = os.path.join(folder_path, 'processed', 'xmcd_projection_blurred')
out_file = os.path.join(folder_path, 'processed', 'projection_movie_blurred1.mp4')
image_files = glob(os.path.join(savedir, '*.png'))

indx = [int(imgf.split('.')[-2].split('_')[-1]) for imgf in image_files]
image_files = [imgf for _, imgf in sorted(zip(indx, image_files))]

# Define the codec and create VideoWriter object
fourcc =cv2.VideoWriter_fourcc('M','J','P','G') # Be sure to use lower case
img = cv2.imread(image_files[0])
out_writer = cv2.VideoWriter(out_file, fourcc, FPS, (img.shape[1], img.shape[0]))

i = 0
for imf in tqdm(image_files):
    img = cv2.imread(imf)
    # append to the video writer
    out_writer.write(img)
    
# Release everything if job is finished
out_writer.release()
cv2.destroyAllWindows()

print("The output video is {}".format(out_file))

100%|██████████████████████████████| 121/121 [00:30<00:00,  3.94it/s]

The output video is D:\Magnumfe\al6-1-1_peem_profile2_001\processed\projection_movie_blurred1.mp4


### Make a tandem video

In [ ]:
import cv2
from tqdm import tqdm
FPS = 10

# folder_path = r"D:\Magnumfe\al6-1-1_peem_profile1"
basesavedir = os.path.join(folder_path, 'processed')
savedirs = [
    os.path.join(basesavedir, 'xmcd_projection_blurred'),
    os.path.join(basesavedir, 'projected_structure')
]
out_file = os.path.join(basesavedir, 'projection_movie_blurred.mp4')

image_files = []
for sd in savedirs:
    image_files_sd = glob(os.path.join(sd, '*.png'))
    indx = [int(imgf.split('.')[-2].split('_')[-1]) for imgf in image_files_sd]
    image_files.append([imgf for _, imgf in sorted(zip(indx, image_files_sd))])

# Define the codec and create VideoWriter object
fourcc =cv2.VideoWriter_fourcc('M','J','P','G') # Be sure to use lower case

img = np.hstack([cv2.imread(imf[0]) for imf in image_files])
out_writer = cv2.VideoWriter(out_file, fourcc, FPS, (img.shape[1], img.shape[0]))

i = 0
for imfs in tqdm(zip(*image_files)):
    img = np.hstack([cv2.imread(imf) for imf in imfs])
    # append to the video writer
    out_writer.write(img)
    
# Release everything if job is finished
out_writer.release()
cv2.destroyAllWindows()

print("The output video is {}".format(out_file))

# Angles study

In [187]:
folder_path = r"D:\Magnumfe\angles_initialisation"

data_folder = os.path.join(folder_path, "data")
mag_files = glob(os.path.join(data_folder, "*.csv"))
msh_file = glob(os.path.join(data_folder, "*.msh"))[0]


msh = Mesh.from_file(msh_file)
raytracing_file = os.path.join(data_folder, "raytracing.npy")
if os.path.exists(raytracing_file):
    raytr = np.load(raytracing_file, allow_pickle=True).item()
    p = raytr.p
else:
    p = get_projection_vector(90, 16)
    # prepare raytracing object
    raytr = RayTracing(msh, p)
    raytr.get_piercings()
    np.save(raytracing_file, raytr, allow_pickle=True)
    
struct = raytr.struct
struct_projected = raytr.struct_projected

100%|█████████████████████████| 84652/84652 [10:29<00:00, 134.49it/s]


In [189]:
magnetisation, _ = load_mesh_magnetisation(mag_files[-1])
xmcd_value = raytr.get_xmcd(magnetisation)
mag_colors = get_struct_face_mag_color(struct, magnetisation)

vis = MeshVisualizer(struct, struct_projected, xmcd_value, struct_colors=mag_colors)

azi=90
center_struct = [-100, 100, 0]
dist_struct = 1e5
center_peem = [600, -5000, 0]
dist_peem = 7e5
vis.show(azi=azi, center=center_peem, dist=dist_peem)

In [190]:
os.path.basename(mag_files[0]).split('.')[0]

'al6-1-1_peem_angle-15'

In [193]:

savedir = os.path.join(folder_path, 'processed', 'xmcd_projection')
savedir_blurred = savedir+'_blurred'
savedir_structure = os.path.join(folder_path, 'processed', 'projected_structure')
for sd in (savedir, savedir_blurred, savedir_structure):
    if not os.path.exists(sd):
        os.makedirs(sd)
    
for mf in tqdm(mag_files):
#     if os.path.exists(save_path):
#         print('continuing')
#         continue
    magnetisation, _ = load_mesh_magnetisation(mf)
    xmcd_value = raytr.get_xmcd(magnetisation)
    mag_colors = get_struct_face_mag_color(struct, magnetisation)
    # update the view
    vis.update_colors(xmcd_value, mag_colors)
    vis.view_projection(center=center_peem, dist=dist_peem)
    file_name = os.path.basename(mf).split('.')[0]+'.png'
    image = vis.save_render(os.path.join(savedir, file_name))
    
    # save the blurred image
    blurred = img2uint(vis.get_blurred_image(desired_background=desired_background))
    imsave(os.path.join(savedir_blurred, file_name), blurred, check_contrast=False)
    
    # save the image of the structure
    vis.view_struct(center=center_struct, dist=dist_struct)
    image = vis.save_render(os.path.join(savedir_structure, file_name))

100%|████████████████████████████████| 14/14 [00:24<00:00,  1.75s/it]


In [195]:
savedir = os.path.join(folder_path, 'processed', 'xmcd_projection_inv')
savedir_blurred = savedir+'_blurred'
savedir_structure = os.path.join(folder_path, 'processed', 'projected_structure_inv')
for sd in (savedir, savedir_blurred, savedir_structure):
    if not os.path.exists(sd):
        os.makedirs(sd)
    
for mf in tqdm(mag_files):
    magnetisation, _ = load_mesh_magnetisation(mf)
    magnetisation *= -1
    xmcd_value = raytr.get_xmcd(magnetisation)
    mag_colors = get_struct_face_mag_color(struct, magnetisation)
    # update the view
    vis.update_colors(xmcd_value, mag_colors)
    vis.view_projection(center=center_peem, dist=dist_peem)
    file_name = os.path.basename(mf).split('.')[0]+'.png'
    image = vis.save_render(os.path.join(savedir, file_name))
    
    # save the blurred image
    blurred = img2uint(vis.get_blurred_image(desired_background=desired_background))
    imsave(os.path.join(savedir_blurred, file_name), blurred, check_contrast=False)
    
    # save the image of the structure
    vis.view_struct(center=center_struct, dist=dist_struct)
    image = vis.save_render(os.path.join(savedir_structure, file_name))

100%|████████████████████████████████| 14/14 [00:25<00:00,  1.85s/it]


In [ ]:
folder_path